<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformers_for_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

# Define the Transformer model for time series data
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_heads, n_layers):
        super(TimeSeriesTransformer, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=n_heads, dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)
        self.fc_out = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.fc_out(x)
        return x

# Example usage
input_dim = 16
hidden_dim = 64
n_heads = 4
n_layers = 3

# Initialize the model
model = TimeSeriesTransformer(input_dim, hidden_dim, n_heads, n_layers)

# Generate random input data (sequence length, batch size, input_dim)
input_data = torch.randn(10, 32, input_dim)

# Get the model output
output = model(input_data)
print(output.shape)  # Expected output shape: (sequence length, batch size, 1)